In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as k_back
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [2]:
# define the AUC metric being used in the competition
# this function can be used in Keras' Sequential Model compilation only
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    k_back.get_session().run(tf.local_variables_initializer())
    return auc

In [3]:
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=',')

In [4]:
train = load_data('data/train_2008.csv', 1)
test = load_data('data/test_2012.csv', 1)

In [5]:
x_train = train[:, :-1]
y_train = train[:, -1]

x_test = test[:,:]

In [6]:
# look at the shapes
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(64667, 382)
(82820, 382)
(64667,)


In [7]:
merged = np.concatenate((x_train, x_test), axis=0)

In [8]:
merged.shape

(147487, 382)

In [9]:
mmscaler = MinMaxScaler()
merged = mmscaler.fit_transform(merged)

In [10]:
# break up into test and train again
x_train = merged[:64667,:]
x_test = merged[64667:,:]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(64667, 382)
(82820, 382)
(64667,)


In [11]:
# define base model
def easy_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [12]:
# evaluate model with standardized dataset
easy_estimator = KerasRegressor(build_fn=easy_model, epochs=10, batch_size=32, verbose=0)

In [13]:
# perform cross validation
easy_kfold = KFold(n_splits=10)
easy_results = cross_val_score(easy_estimator, x_train, y_train, cv=easy_kfold)
print("Results: %.2f (%.2f) MSE" % (easy_results.mean(), easy_results.std()))

Results: -0.16 (0.00) MSE


In [14]:
# fit to train, predict both train and test
easy_estimator.fit(x_train, y_train)
easy_y_train = easy_estimator.predict(x_train)
easy_y_test = easy_estimator.predict(x_test)

In [15]:
print (easy_y_train.shape)
print (easy_y_test.shape)

(64667,)
(82820,)


In [16]:
# look at numbers of predictions out of bounds
print (np.sum(np.array(easy_y_train) < 0))
print (np.sum(np.array(easy_y_train) > 1))
print (np.sum(np.array(easy_y_test) < 0))
print (np.sum(np.array(easy_y_test) > 1))

582
51
60
132


In [18]:
# reshape to (x, 1) and normalize to (0,1)
norm_y_train = easy_y_train.reshape(64667, 1)
norm_y_test = easy_y_test.reshape(82820, 1)

# normalize output data
norm_y_train = mmscaler.fit_transform(norm_y_train)
norm_y_test = mmscaler.fit_transform(norm_y_test)

# check shapes
print (norm_y_train.shape)
print (norm_y_test.shape)

(64667, 1)
(82820, 1)


In [19]:
roc_auc_score(y_train, norm_y_train)

0.7827913154916586

In [20]:
# output to file
ids = test[:,:1]
ids = ids.astype(int)
predictions = norm_y_test.astype(float)
out = np.concatenate((ids, predictions), axis=1)

np.savetxt('clean_submission_2012.csv', out, delimiter=',', fmt='%1.10f', header = 'id,target')